#### Series de tiempo

se tratan de un conjunto de observaciones históricas almacenadas de forma secuencial que representan un proceso de cambios en los valores conforme avanza el tiempo, algunos ejemplos de series temporales son el mercado finanziero, el clima o el histórico de ventas en una empresa. La predicción del futuro a partir de indicios es uno de los principales objetivos junto con el análisis descriptivo e inferencial en las series de tiempo, algunos métodos como el ingenuo, la media, la deriva y la media móvil simple se consideran muy básicos y que no aportan variabilidad en los resultados. Sin embargo, el aprendizaje automático también es implementado en el pronóstico y existen diferentes algoritmos que se ajustan y reconocen patrones de ciertas series temporales.

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input


df = pd.read_csv("data/bitcoin_usd.csv")
df["date"] = pd.to_datetime(df["date"], format='%Y-%m-%d')
df.sort_values(by="date", inplace=True)
df

#### Análisis exploratorio de series temporales

Nivel: media estadística de la serie.

Estacionariedad: componente en series cuyas medidas(nivel y desviación éstandar) no cambian con el tiempo.

Tendencia: dirección general y el cambio a largo plazo del nivel de la serie.

Estacionalidad: picos y valles que se repiten periódicamente(diarios, semanales, anuales, etc), es decir, siguiendo un patrón.

Ruido: al igual que un conjunto de datos convencional, sufren de errores aleatorios e imprecisiones que no tienen un origen concreto.

In [ ]:
df.set_index("date", inplace=True)

fig, ax = plt.subplots(5,1, figsize=(9,18))

def trend(serie, index):
    ax[index].plot(df[serie], label="Serie temporal")
    ax[index].plot(seasonal_decompose(df[serie], model="aditive").trend, label="Tendencia")
    ax[index].set_ylabel(serie)
    ax[index].set_xlabel("dates")
    ax[index].legend()
    ax[index].grid("on")

trend("volume", 0)
trend("open_USD", 1)
trend("close_USD", 2)
trend("high_USD", 3)
trend("low_USD", 4)

plt.subplots_adjust(hspace=0.4)
plt.show()

#### Observaciones 

a excepción de la variable 'volume', todas las demás series presentan una tendencia en aumento con el paso del tiempo, sobre todo de forma notable en el último período. También, podemos anular una posible estacionariedad debido al constante cambio en la media y desvación éstandar de la serie, un método utilizado es la prueba de Dickey-Fuller en estas suposiciones pero en este caso es muy notoria visualmente la no unitariedad.

______________________________________________________________________________________________________________________________

#### Análisis de estacionalidad 

Autocorrelación: mide la correlación lineal entre una serie de tiempo y una versión de esta con un determinado retardo, es decir, sin los valores más recientes.

Función de autocorrelación: diagrama que muestra un conjunto de autocorrelaciones con sus respectivos retardos, mediante observemos los cambios de la figura podremos estimar un posible patrón estacional.

In [ ]:
fig, ax = plt.subplots(5,1, figsize=(20,65))

def correlogram(serie, index):
    plot_acf(df[serie], lags=df.shape[0] - 400, ax=ax[index])
    ax[index].set_title(serie)
    ax[index].set_ylabel("Autocorrelación")
    ax[index].set_xlabel("Retardo(K)")

correlogram("volume", 0)
correlogram("open_USD", 1)
correlogram("close_USD", 2)
correlogram("high_USD", 3)
correlogram("low_USD", 4)

plt.subplots_adjust(hspace=0.3)
plt.show()

#### Observaciones 
al observar los diferentes corregloramas no podemos asimilar una estacionalidad estándar clara, también los valores negativos explican cierto ruido o cambios bruscos en algunas partes del progreso. En base a las funciones de autocorrelación, en las series que representan diferentes avanzes financieros se pueden determinar ciclos en los valores, estos a diferencia de la estacionalidad los máximos y mínimos suelen tener períodos irregulares, pueden ser causados por factores internos y su duración es más prolongada.

______________________________________________________________________________________________________________________________

#### Dashboard de forecasting realizado por un modelo adecuado a la serie de tiempo

In [ ]:
df.reset_index(inplace=True)

app = dash.Dash(__name__)
app.layout =  html.Div(id="body", className="e7_body", children=[
    html.H1("Forecasting", id="title", className="e7_title"),
    dcc.Dropdown(id="dropdown", className="e7_dropdown",
                        options=[
                            {"label":"Vólumen de trading","value":"volume"},
                            {"label":"Precio de apertura","value":"open_USD"},
                            {"label":"Precio de cierre","value":"close_USD"},
                            {"label":"Precio más bajo","value":"low_USD"},
                            {"label":"Precio más alto","value":"high_USD"},

                        ],
                        value="volume",
                        multi=False,
                        clearable=False),
    dcc.Graph(id="forecasting", figure={}, className="e7_graph")
])

@app.callback(
    Output(component_id="forecasting",component_property="figure"),
    [Input(component_id="dropdown",component_property="value")]
)

def update_graph(slct_var):
    
    if slct_var == "volume":
        TES = ExponentialSmoothing(df[slct_var], seasonal="add", seasonal_periods=255)
    elif slct_var == "high_USD":
        TES = ExponentialSmoothing(df[slct_var], trend="add", seasonal="add", seasonal_periods=335)
    else:
        TES = ExponentialSmoothing(df[slct_var], trend="add", seasonal="add", seasonal_periods=325)
    
    model = TES.fit()
            
    new_dates = list(pd.date_range('2021-01-31','2021-03-31'))
    predicts = model.forecast(60)
    
    forecasting = go.Figure()
    forecasting.add_trace(go.Scatter(x=df["date"], y=df[slct_var], mode="lines", name="Serie"))
    forecasting.add_trace(go.Scatter(x=new_dates, y=predicts, mode="lines", name="Pronóstico"))
    forecasting.update_layout(title_text="Pronóstico de los próximos 2 meses")
    forecasting.update_yaxes(title_text=slct_var)

    return forecasting

if __name__ == "__main__":
    app.run_server(debug=False)

#### Suavización Exponencial

dentro de los algoritmos de series temporales se encuentran ARIMA, SARIMA o el que se implementó en en este ejemplo, su método se basa precisamente en suavizar las fluctuaciones de la serie a través de una combinación ponderada de valores anteriores, los parámetros que utiliza son el factor de suavización(Valor Alpha) y el Nivel inicial(0). Dependiendo del comportamiento de la serie el modelo puede ir variando y añadiendo más componentes, algunos variaciones en el Suavizado Exponencial son: Suavizado Simple, Tendencia Lineal de Holt, Tendencia Amortiguada, Estacional Simple o Aditivo de Winters.

Suavización Exponencial Triple o Estacional Simple: modelo apropiado para series de tiempo con tendencia y un efecto estacional que es constante a lo largo del tiempo, sus parámetros de suavizado son el nivel, la tendencia y la estacionalidad, además se requiere de la espicificación sobre cada cuanto es el período en que se da el patrón. En este caso, es importante tener en cuenta la complejidad y la volatidad del mercado del Bitcoin, por lo tanto es más considerable que las diferentes series sean cíclicas, es decir que los patrones son más complejos y no necesariamente periódicos, esto provoca que el modelo de suavizado(SES), al ser sensible a los hiperparámetros, tenga dificultades para capturalos y sea más propenso a producir errores de predicción.